# TA-Lib Tutorial

"TA-Lib is widely used by trading software developers requiring to perform technical analysis of financial market data."  
We will show how to use them here, in the context of pinkfish.

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# use future imports for python 3.x forward compatibility
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import division
from __future__ import absolute_import

# other imports
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import talib
from talib.abstract import *
from talib import MA_Type
import itable

# project imports
import pinkfish as pf

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Some global data

In [4]:
symbol = 'SPY'
start = datetime.datetime(2018, 1, 1)
end = datetime.datetime.now()

Fetch symbol data from cache, if available.

In [5]:
ts = pf.fetch_timeseries(symbol)

In [6]:
ts.tail()

,high,low,open,close,volume,adj_close
date,,,,,,
2020-04-20,286.79,281.35,282.61,281.59,100109300.00,281.59
2020-04-21,278.04,272.02,276.73,273.04,126385700.00,273.04
2020-04-22,281.00,276.91,278.35,279.10,93524600.00,279.10
2020-04-23,283.94,278.75,280.49,279.08,104709700.00,279.08
2020-04-24,283.70,278.50,280.73,282.97,85063200.00,282.97


Select timeseries between start and end.

In [7]:
ts = pf.select_tradeperiod(ts, start, end)

In [8]:
ts.head()

,high,low,open,close,volume,adj_close
date,,,,,,
2017-01-03,225.83,223.88,225.04,225.24,91366500.00,211.55
2017-01-04,226.75,225.61,225.62,226.58,78744400.00,212.81
2017-01-05,226.58,225.48,226.27,226.40,78379000.00,212.64
2017-01-06,227.75,225.90,226.53,227.21,71559900.00,213.40
2017-01-09,227.07,226.42,226.91,226.46,46939700.00,212.70


### Get info about TA-Lib

In [9]:
print('There are {} TA-Lib functions!'.format(len(talib.get_functions())))

There are 158 TA-Lib functions!


Here is a complete listing of the functions by group:

In [10]:
for group, funcs in talib.get_function_groups().items():
    print(group)
    print('-----------------------------------------')
    for func in funcs:
        f = Function(func)
        print('{} - {}'.format(func, f.info['display_name']))
    print()

Cycle Indicators
-----------------------------------------
HT_DCPERIOD - Hilbert Transform - Dominant Cycle Period
HT_DCPHASE - Hilbert Transform - Dominant Cycle Phase
HT_PHASOR - Hilbert Transform - Phasor Components
HT_SINE - Hilbert Transform - SineWave
HT_TRENDMODE - Hilbert Transform - Trend vs Cycle Mode

Math Operators
-----------------------------------------
ADD - Vector Arithmetic Add
DIV - Vector Arithmetic Div
MAX - Highest value over a specified period
MAXINDEX - Index of highest value over a specified period
MIN - Lowest value over a specified period
MININDEX - Index of lowest value over a specified period
MINMAX - Lowest and highest values over a specified period
MINMAXINDEX - Indexes of lowest and highest values over a specified period
MULT - Vector Arithmetic Mult
SUB - Vector Arithmetic Substraction
SUM - Summation

Math Transform
-----------------------------------------
ACOS - Vector Trigonometric ACos
ASIN - Vector Trigonometric ASin
ATAN - Vector Trigonometric AT

### Get info about a specific TA-Lib function

There are 2 different API that are available with talib, namely Function API and Abstract API.  For the Function API, you pass in a price series.  For the Abstract API, you pass in a collection of named inputs: 'open', 'high', 'low', 'close', and 'volume'.  One or more of these may be used as defaults, but can be changed with the 'price' parameter.  

Print the function instance to get documentation.  We see that SMA has the parameter 'timeperiod' with default '30'.  The input_arrays can be a dataframe with columns named 'open', 'high', 'low', 'close', and 'volume'.

In [11]:
print(SMA)

SMA([input_arrays], [timeperiod=30])

Simple Moving Average (Overlap Studies)

Inputs:
    price: (any ndarray)
Parameters:
    timeperiod: 30
Outputs:
    real


More information is available through the 'info' property.  We observe here that the default price used is 'close'.  This can be changed by setting 'price' in the function call, e.g. price='open'.

In [12]:
print(SMA.info)

{'name': 'SMA', 'group': 'Overlap Studies', 'display_name': 'Simple Moving Average', 'function_flags': ['Output scale same as input'], 'input_names': OrderedDict([('price', 'close')]), 'parameters': OrderedDict([('timeperiod', 30)]), 'output_flags': OrderedDict([('real', ['Line'])]), 'output_names': ['real']}


If we just want to see the inputs, we can print the input_names property.

In [13]:
print(SMA.input_names)

OrderedDict([('price', 'close')])


If we just want to see the parameters, we can print the paramters property.

In [14]:
print(SMA.parameters)

OrderedDict([('timeperiod', 30)])


If we just want to see the outputs, we can print the output_names property.

In [15]:
print(SMA.output_names)

['real']


### Create a technical indicator using talib

Create technical indicator: SMA (using defaults: timeperiod=30, price='close')

In [16]:
sma = SMA(ts)
sma.tail()

date
2020-04-20   260.54
2020-04-21   260.50
2020-04-22   260.19
2020-04-23   260.35
2020-04-24   261.51
dtype: float64

Create technical indicator: SMA (using: timeperiod=200, price='close')

In [17]:
sma200 = SMA(ts, timeperiod=200)
sma200.tail()

date
2020-04-20   300.76
2020-04-21   300.63
2020-04-22   300.54
2020-04-23   300.45
2020-04-24   300.37
dtype: float64

Create technical indicator: SMA (using: timeperiod=200, price='open')

In [18]:
sma200 = SMA(ts, timeperiod=200, price='open')
sma200.tail()

date
2020-04-20   300.63
2020-04-21   300.53
2020-04-22   300.43
2020-04-23   300.36
2020-04-24   300.27
dtype: float64

### Add a technical indicator to a pinkfish timeseries

In [19]:
ts['sma200'] = sma200
ts.tail()

,high,low,open,close,volume,adj_close,sma200
date,,,,,,,
2020-04-20,286.79,281.35,282.61,281.59,100109300.00,281.59,300.63
2020-04-21,278.04,272.02,276.73,273.04,126385700.00,273.04,300.53
2020-04-22,281.00,276.91,278.35,279.10,93524600.00,279.10,300.43
2020-04-23,283.94,278.75,280.49,279.08,104709700.00,279.08,300.36
2020-04-24,283.70,278.50,280.73,282.97,85063200.00,282.97,300.27


### Try another one

Commodity Channel Index

In [20]:
print(CCI)

CCI([input_arrays], [timeperiod=14])

Commodity Channel Index (Momentum Indicators)

Inputs:
    prices: ['high', 'low', 'close']
Parameters:
    timeperiod: 14
Outputs:
    real


In [21]:
print(CCI.input_names)

OrderedDict([('prices', ['high', 'low', 'close'])])


In [22]:
print(CCI.parameters)

OrderedDict([('timeperiod', 14)])


In [23]:
cci = CCI(ts)
ts['cci'] = cci
ts.tail()

,high,low,open,close,volume,adj_close,sma200,cci
date,,,,,,,,
2020-04-20,286.79,281.35,282.61,281.59,100109300.00,281.59,300.63,83.01
2020-04-21,278.04,272.02,276.73,273.04,126385700.00,273.04,300.53,25.93
2020-04-22,281.00,276.91,278.35,279.10,93524600.00,279.10,300.43,49.44
2020-04-23,283.94,278.75,280.49,279.08,104709700.00,279.08,300.36,57.24
2020-04-24,283.70,278.50,280.73,282.97,85063200.00,282.97,300.27,64.36


### Now for something a little more difficult

Bollinger Bands

In [24]:
print(BBANDS)

BBANDS([input_arrays], [timeperiod=5], [nbdevup=2], [nbdevdn=2], [matype=0])

Bollinger Bands (Overlap Studies)

Inputs:
    price: (any ndarray)
Parameters:
    timeperiod: 5
    nbdevup: 2
    nbdevdn: 2
    matype: 0 (Simple Moving Average)
Outputs:
    upperband
    middleband
    lowerband


In [25]:
print(BBANDS.input_names)

OrderedDict([('price', 'close')])


In [26]:
print(BBANDS.parameters)

OrderedDict([('timeperiod', 5), ('nbdevup', 2), ('nbdevdn', 2), ('matype', 0)])


Print the available moving average types

In [27]:
attributes = [attr for attr in dir(MA_Type) 
              if not attr.startswith('__')]
attributes

['DEMA', 'EMA', 'KAMA', 'MAMA', 'SMA', 'T3', 'TEMA', 'TRIMA', 'WMA', '_lookup']

In [28]:
MA_Type.__dict__

{'_lookup': {0: 'Simple Moving Average',
  1: 'Exponential Moving Average',
  2: 'Weighted Moving Average',
  3: 'Double Exponential Moving Average',
  4: 'Triple Exponential Moving Average',
  5: 'Triangular Moving Average',
  6: 'Kaufman Adaptive Moving Average',
  7: 'MESA Adaptive Moving Average',
  8: 'Triple Generalized Double Exponential Moving Average'}}

In [29]:
print(MA_Type[MA_Type.DEMA])

Double Exponential Moving Average


Set timeperiod=20 and matype=MA_Type.EMA

In [30]:
#upper, middle, lower = BBANDS(ts, timeperiod=20, matype=MA_Type.EMA)
#(for some reason, the abstract API doesn't work for BBANDS, so use the function API)

upper, middle, lower = talib.BBANDS(ts.close, timeperiod=20, matype=MA_Type.EMA)
ts['upper'] = upper; ts['middle'] = middle; ts['lower'] = lower
ts.tail()

,high,low,open,close,volume,adj_close,sma200,cci,upper,middle,lower
date,,,,,,,,,,,
2020-04-20,286.79,281.35,282.61,281.59,100109300.00,281.59,300.63,83.01,304.00,271.29,238.59
2020-04-21,278.04,272.02,276.73,273.04,126385700.00,273.04,300.53,25.93,298.74,271.46,244.18
2020-04-22,281.00,276.91,278.35,279.10,93524600.00,279.10,300.43,49.44,298.04,272.19,246.34
2020-04-23,283.94,278.75,280.49,279.08,104709700.00,279.08,300.36,57.24,297.37,272.84,248.32
2020-04-24,283.70,278.50,280.73,282.97,85063200.00,282.97,300.27,64.36,298.80,273.81,248.82
